In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import matplotlib.pyplot as plt
import plotly.express as px

/Users/aditkotwal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
headers = {"User-Agent": "adit29my@gmail.com"} 

## Fetching historical data

In [4]:
def getCIKNumber(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

In [5]:
def getSubmissionData(ticker, headers=headers):
    cik = getCIKNumber(ticker)
    headers = headers
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()
    
    return pd.DataFrame(company_json["filings"]["recent"])

In [6]:
def getFilteredFilings(ticker, form, headers=headers):
    company_filings_df = getSubmissionData(
        ticker, headers=headers
    )
    if (form=='ten_k'):
        df = company_filings_df[company_filings_df["form"] == "10-K"]
    elif (form=='ten_q'):
        df = company_filings_df[company_filings_df["form"] == "10-Q"]
    
    df = df.set_index("reportDate")
    accession_df = df["accessionNumber"]
    return accession_df    

In [7]:
def getFacts(ticker, headers=headers):
    cik = getCIKNumber(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    company_facts = requests.get(url, headers=headers).json()
    return company_facts

In [8]:
def getFactsDF(ticker, headers=headers):
    facts = getFacts(ticker, headers)
    us_gaap_data = facts["facts"]["us-gaap"]
    df_data = []
    for fact, details in us_gaap_data.items():
        for unit in details["units"]:
            for item in details["units"][unit]:
                row = item.copy()
                row["fact"] = fact
                df_data.append(row)

    df = pd.DataFrame(df_data)
    df["end"] = pd.to_datetime(df["end"])
    df["start"] = pd.to_datetime(df["start"])
    df = df.drop_duplicates(subset=["fact", "end", "val"])
    df.set_index("end", inplace=True)
    labels_dict = {fact: details["label"] for fact, details in us_gaap_data.items()}
    return df, labels_dict

In [9]:
def getAnnualFacts(ticker, headers=headers):
    accession_nums = getFilteredFilings(ticker, form='ten_k')
    df, label_dict = getFactsDF(ticker, headers)
    ten_k = df[df["accn"].isin(accession_nums)]
    ten_k = ten_k[ten_k.index.isin(accession_nums.index)]
    pivot = ten_k.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [10]:
def getQuarterlyFacts(ticker, headers=headers):
    accession_nums = getFilteredFilings(ticker, form='ten_q')
    df, label_dict = getFactsDF(ticker, headers)
    ten_q = df[df["accn"].isin(accession_nums)]
    ten_q = ten_q[ten_q.index.isin(accession_nums.index)].reset_index(drop=False)
    ten_q = ten_q.drop_duplicates(subset=["fact", "end"], keep="last")
    pivot = ten_q.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [11]:
def getHistoricalData(ticker):
    quarterlyDF = getQuarterlyFacts(ticker)
    annualDF = getAnnualFacts(ticker)

    # Step 1: Find common rows
    common_indices = quarterlyDF.index.intersection(annualDF.index)

    # Step 2: Subset DataFrames to common rows
    Df1_common = quarterlyDF.loc[common_indices]
    Df2_common = annualDF.loc[common_indices]

    # Step 3: Combine the two DataFrames
    combined_df = pd.concat([Df1_common, Df2_common], axis=1)

    # Step 4: Sort the columns by date
    combined_df = combined_df.reindex(sorted(combined_df.columns), axis=1)

    return combined_df

## Financial Ratios

In [14]:
def addColumns(df, ratioDictionary):
    for col_name, (required_cols, operation) in ratioDictionary.items():
        if all(col in df.columns for col in required_cols):
            df[col_name] = operation(df)
        else:
            print(f"Skipping '{col_name}': Missing required columns {required_cols}.")
    return df

In [15]:
def calculateRatios(dataDf):
    
    df = pd.DataFrame(dataDf).T
    df.index = pd.to_datetime(df.index)

    if 'Revenue, Net (Deprecated 2018-01-31)' in df.columns:
        df['Effective Revenue'] = df['Revenue, Net (Deprecated 2018-01-31)'].fillna(df['Revenue from Contract with Customer, Excluding Assessed Tax'])
    else:
        df['Effective Revenue'] = df['Revenue from Contract with Customer, Excluding Assessed Tax']
    
    df['Operating Margin Ratio'] = df['Operating Income (Loss)'] / df['Effective Revenue']
    ratioDictionary = {
        'Gross Margin Ratio': (['Gross Profit', 'Effective Revenue'], lambda df: df['Gross Profit'] / df['Effective Revenue']),
        'Operating Margin Ratio': (['Operating Income (Loss)', 'Effective Revenue'], lambda df: df['Operating Income (Loss)'] / df['Effective Revenue']),
    }

    df_with_ratios = addColumns(df, ratioDictionary)
    return df_with_ratios